<a href="https://colab.research.google.com/github/Mukiti-Ryan/IS_Project_Presentation/blob/main/Churn_Model_(Work_TFDF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_decision_forests -q
!pip install keras-tuner -q

import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf
import keras_tuner as kt
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.4 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IS Project 2 Datasets/churnmodel.csv')

df.shape

<ipython-input-2-a86b2ceeef59>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/IS Project 2 Datasets/churnmodel.csv')


(1000000, 16)

In [ ]:
df.head()

,Unnamed: 0,CD_ACCOUNT,STATUS2,DAYS_ACC_OPEN,CD_TYPE,VL_CREDIT_RECENCY,VL_DEBIT_RECENCY,VL_TENOR,AVGCREDITTURNOVER_LY,AVGDEBITTURNOVER_LY,AVGCREDIT_TRANS_LY,AVGDEBIT_TRANS_LY,AVGCREDITTURNOVER_LY2,AVGDEBITTURNOVER_LY2,AVGCREDIT_TRANS_LY2,AVGDEBIT_TRANS_LY2
0,0,1100000000000.0,NOT CHURNED,6173,1062,21.0,0.0,6170.0,438197.26,-474788.26,3.08,93.66,720779.28,-748023.72,6.33,133.41
1,1,1100000000000.0,CHURNED,6163,1062,313.0,0.0,6118.0,1666.66,-6590.41,0.16,1.25,1666.66,-9322.83,0.16,3.08
2,2,1100000000000.0,CHURNED,6152,1062,2126.0,1949.0,6132.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,3,1100000000000.0,CHURNED,6124,1062,982.0,0.0,5770.0,0.00,-1220.00,0.00,0.41,0.00,-2102.00,0.00,0.91
4,4,1100000000000.0,NOT CHURNED,6117,1062,0.0,0.0,6091.0,4126.41,-200.50,1.08,3.66,7935.41,-395.00,2.08,7.50


In [ ]:
if 'Unnamed: 0' in df.columns:
  df = df.drop(columns = ['Unnamed: 0'])

In [ ]:
df.drop(['CD_ACCOUNT'], axis = 1, inplace = True)

df.shape

(1000000, 14)

In [ ]:
neg_counts = (df.select_dtypes(include = np.number) < 0).sum()

if neg_counts.any():
  print("Columns with negative values: ")
  print(neg_counts[neg_counts != 0])

  print(f"Total number of negative values: {neg_counts.sum()}")
else:
  print("There are no negative values in the dataset")

Columns with negative values: 
VL_CREDIT_RECENCY          1136
VL_DEBIT_RECENCY             98
VL_TENOR                  10212
AVGCREDITTURNOVER_LY         22
AVGDEBITTURNOVER_LY      746136
AVGCREDIT_TRANS_LY            6
AVGCREDITTURNOVER_LY2        17
AVGDEBITTURNOVER_LY2     792223
AVGCREDIT_TRANS_LY2           5
dtype: int64
Total number of negative values: 1549855


In [ ]:
numeric_cols = df.select_dtypes(include = [int, float]).columns
df[numeric_cols] = df[numeric_cols].abs()

In [ ]:
df.describe()

,DAYS_ACC_OPEN,CD_TYPE,VL_CREDIT_RECENCY,VL_DEBIT_RECENCY,VL_TENOR,AVGCREDITTURNOVER_LY,AVGDEBITTURNOVER_LY,AVGCREDIT_TRANS_LY,AVGDEBIT_TRANS_LY,AVGCREDITTURNOVER_LY2,AVGDEBITTURNOVER_LY2,AVGCREDIT_TRANS_LY2,AVGDEBIT_TRANS_LY2
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1.000000e+06,1.000000e+06,1000000.000000,1000000.000000,1.000000e+06,1.000000e+06,1000000.000000,1000000.000000
mean,2284.078102,1062.066047,1253.549290,1979.869468,2382.946083,6.199433e+04,6.279789e+04,2.469595,4.593288,2.075195e+05,2.086299e+05,4.622182,8.833637
std,1737.424068,0.248364,4144.024187,5478.787502,2439.527684,8.613043e+06,8.593854e+06,49.061543,17.684722,9.422977e+07,9.423014e+07,90.504225,29.221068
min,20.000000,1062.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,762.000000,1062.000000,0.000000,0.000000,721.000000,0.000000e+00,0.000000e+00,0.000000,0.080000,2.080000e+00,3.100000e-01,0.080000,0.250000
50%,1972.000000,1062.000000,26.000000,0.000000,1862.000000,2.416600e+02,1.625000e+02,0.330000,0.580000,1.532080e+03,1.333330e+03,0.580000,1.160000
75%,3461.000000,1062.000000,392.000000,378.000000,3456.000000,9.986177e+03,1.022263e+04,0.910000,3.910000,2.098320e+04,2.114132e+04,1.660000,7.410000
max,44655.000000,1063.000000,19406.000000,19407.000000,44653.000000,6.375964e+09,6.375980e+09,25433.250000,12109.330000,9.306696e+10,9.306696e+10,47800.660000,15476.660000


In [ ]:
label = "STATUS2"

classes = list(df[label].unique())
print(f"Label Classes: {classes}")

df[label] = df[label].map(classes.index)

Label Classes: ['NOT CHURNED', 'CHURNED']


In [ ]:
np.random.seed(1)

test_indices = np.random.rand(len(df)) < 0.1
df_train = df[~test_indices]
df_test = df[test_indices]

print("Training example: ", len(df_train))
print("Testing example: ", len(df_test))

Training example:  900022
Testing example:  99978


In [ ]:
tf_train = tfdf.keras.pd_dataframe_to_tf_dataset(df_train, label = label)
model = tfdf.keras.CartModel()
model.fit(tf_train)

Use /tmp/tmp8jco7ipa as temporary training directory
Reading training dataset...
Training dataset read in 0:00:23.586583. Found 900022 examples.
Training model...
Model trained in 0:00:02.021702
Compiling model...
Model compiled.


In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth = 10)

In [ ]:
model.compile("accuracy")

print("Train evaluation: ", model.evaluate(tf_train, return_dict = True))

tf_test = tfdf.keras.pd_dataframe_to_tf_dataset(df_test, label = label)
print("Test evaulation: ", model.evaluate(tf_test, return_dict = True))

901/901 [==============================] - 13s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Train evaluation:  {'loss': 0.0, 'accuracy': 0.9999955296516418}
100/100 [==============================] - 1s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test evaulation:  {'loss': 0.0, 'accuracy': 0.9999899864196777}


In [ ]:
def build_model(hp):
  model = tfdf.keras.CartModel(
      min_examples = hp.Choice("min_examples", [1, 2, 4, 10]),
      validation_ratio = hp.Choice("validation_ratio", [0.0, 0.05, 0.10]),
  )

  model.compile("accuracy")
  return model

tuner = kt.RandomSearch(
    build_model,
    objective = "val_accuracy",
    max_trials = 10,
    directory = "/tmp/tuner",
    project_name = "tune_cart")

tuner.search(x = tf_train, validation_data = tf_test)
best_model = tuner.get_best_models()[0]

print("The Best hyperparameters are: ", tuner.get_best_hyperparameters()[0].values)

Trial 10 Complete [00h 00m 21s]
val_accuracy: 0.9998999834060669

Best val_accuracy So Far: 0.9999899864196777
Total elapsed time: 00h 03m 45s
Use /tmp/tmpcvlvoeph as temporary training directory
The Best hyperparameters are:  {'min_examples': 10, 'validation_ratio': 0.1}


In [ ]:
model = tfdf.keras.CartModel(min_examples = 4, validation_ratio = 0.0)
model.fit(tf_train)

model.compile("accuracy")
print("Test evaluation: ", model.evaluate(tf_test, return_dict = True))

Use /tmp/tmp1u3evckm as temporary training directory
Reading training dataset...
Training dataset read in 0:00:11.126334. Found 900022 examples.
Training model...
Model trained in 0:00:02.208004
Compiling model...
Model compiled.
100/100 [==============================] - 1s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test evaluation:  {'loss': 0.0, 'accuracy': 0.9999899864196777}


In [ ]:
import matplotlib.pyplot as plt

tfdf.model_plotter.plot_model_in_colab(model, max_depth = 10)
plt.savefig("decision_tree_visualization.png")

<Figure size 640x480 with 0 Axes>

In [ ]:
# Specify the tree index you want to visualize
tree_idx = 0

# Plot the decision tree
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=tree_idx)
plt.show()

In [ ]:
plt.savefig("decision_tree_visualization.png")

<Figure size 640x480 with 0 Axes>

In [ ]:
with open('model.pkl', 'wb') as f:
  pickle.dump(model, f)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
...vars
......0
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-02-27 06:33:55          343
metadata.json                                  2023-02-27 06:33:55           64
variables.h5                                   2023-02-27 06:33:55        13064


In [ ]:
from google.colab import files
files.download('model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>